In [1]:
import torch

print("Torch version:", torch.__version__)
print("Is Metal available:", "Yes" if torch.backends.mps.is_available() else "No")
print("Is GPU available:", "Yes" if torch.cuda.is_available() else "No")

ModuleNotFoundError: No module named 'torch'